# v9fs performance studies

First thing we need to do is start a qemu with a kernel.  The versions of both of these matter if what we are doing is qemu-virtio as transport (and probably also have an impact on qemu network performance)

TODO:
- [ ] TASK: add build configuraton for [qemu | kernel | u-root | etc]

# Setup Test Environment

The following command starts qemu in the background.  You can optionally set the QEMU command line and KERNEL to use as environment variables.
You must be setup for kvm and the user must have permission to start kvm instances without sudo (usually by being in group kvm).

Okay, so this all works ... but we don't have any nice way to cleanup, so we'll have to work that out at some point.

In [ ]:
QEMU = "../qemu/build/qemu-system-x86_64"
KERNEL = "../build/arch/x86_64/boot/bzImage"


In [ ]:
!sg kvm -c '{QEMU} -kernel \
    {KERNEL} \
	-cpu  max \
    -s   \
    -enable-kvm \
    -m 8192m \
    -machine q35  \
    -initrd /tmp/initramfs.linux_amd64.cpio \
    -object rng-random,filename=/dev/urandom,id=rng0 \
    -device virtio-rng-pci,rng=rng0 \
    -device vhost-vsock-pci,guest-cid=42 \
    -device virtio-net-pci,netdev=n1 \
    -netdev user,id=n1,hostfwd=tcp:127.0.0.1:17010-:17010,net=192.168.1.0/24,host=192.168.1.1 \
    -debugcon file:debug.log -global isa-debugcon.iobase=0x402 \
    -fsdev local,security_model=passthrough,id=fsdev0,path=/tmp \
    -device virtio-9p-pci,id=fs0,fsdev=fsdev0,mount_tag=hostshare \
    -daemonize'

# Run a test

Okay, so now we need some simple tests to run in this environment.  We'll do this all over cpu.  Most operations will be stored in bench.py, but we'll configure them here



In [2]:
%load_ext autoreload
%autoreload 2

In [38]:
#!/usr/bin/python
from bench import BenchTarget, BenchProgress, bandwidth
from IPython.display import clear_output, display
import logging
import pandas as pd

npfs = BenchTarget("npfs", ["../npfs/fs/npfs", "-p", "5640", "-w", "8", "-s"], "/mnt/9/test", "netfstab")
p9ufs = BenchTarget("p9ufs",["../p9/cmd/p9ufs/p9ufs","0.0.0.0:5640"], "/mnt/9/test", "netfstab")
virtio = BenchTarget("virtio","","/mnt/9/test", "virtiofstab")
cpud = BenchTarget("cpu", "", "./test", "fstab")

# testcfgs = [ virtio, npfs, p9ufs, cpud ]
blocksizes = [ 131072, 65536, 32768, 16384, 8192, 4096, 1024, 512 ]
testcfgs = [ virtio ]

def stepDesc( notes, s, t):
    clear_output(wait=True)
    print( notes + "["+str(s)+"/"+str(t)+"]")

prog = BenchProgress(len(testcfgs)*len(blocksizes)*5, stepDesc)

logger = logging.getLogger()
logging.basicConfig(filename="log.txt", level=logging.DEBUG)

results = bandwidth( testcfgs, blocksizes, 5, 131072*2, "notebook test", prog, logger) 

clear_output(wait=True)
display(results)


,Run Config,Target,Blocksize,Iteration:,Bandwidth
0,96,virtio,131072,0,6.5
1,96,virtio,131072,1,6.6
2,96,virtio,131072,2,6.5
3,96,virtio,131072,3,6.7
4,96,virtio,131072,4,6.7
5,96,virtio,65536,0,7.6
6,96,virtio,65536,1,7.2
7,96,virtio,65536,2,7.2
8,96,virtio,65536,3,7.0
9,96,virtio,65536,4,7.2
